In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import os

if os.getcwd().split("/")[-1] != "Road-Segmentation-ML":
    os.chdir("..")
print("CWD:", os.getcwd())

In [ ]:
import torch
from omegaconf import OmegaConf
from utils import set_seeds
from train_utils import prepare_data, prepare_model, prepare_optimizer, train

In [ ]:
# create folder models if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

In [ ]:
args = OmegaConf.create(
    dict(
        # General
        seed=0,
        # Data folders
        train_image_folders=[
            "datasets/train/images/",
            # "datasets/massachusetts_384/images/",
            # "datasets/kaggle/images/",
        ],  # add Massachusetts here
        train_gt_folders=[
            "datasets/train/groundtruth/",
            # "datasets/massachusetts_384/groundtruth/",
            # "datasets/kaggle/groundtruth/",
        ],  # add Massachusetts here
        val_image_folders=["datasets/validation/images/"],
        val_gt_folders=["datasets/validation/groundtruth/"],
        weighted_random_sampler=False,
        # Data tranforms
        # random_resized_crop: crop a random portion of image and resize it to a given size
        random_resized_crop=False,
        output_size=(400, 400),  # expected output size of the crop, for each edge.
        input_size=(400),  # resize
        random_resized_crop_scale=(0.5, 0.5),
        # random_horizontal_flip: randomly flip the image horizontally with a given probability
        random_horizontal_flip=True,
        # random_vertical_flip: randomly flip the image vertically with a given probability
        random_vertical_flip=True,
        # random_rotation: randomly rotate the image with a given probability
        random_rotation=True,
        degrees=10,  # range of degrees to select from
        # color_jitter: randomly change the brightness, contrast and saturation of an image
        color_jitter=False,
        brightness=0.1,  # how much to jitter brightness.
        contrast=0.1,  # how much to jitter contrast.
        saturation=0.1,  # how much to jitter saturation.
        hue=0.1,  # how much to jitter hue.
        # normalization
        normalization=True,  # TODO: Should it always be True?
        normalization_flag="A",  # "A" for AIcrowd, "AM" for AIcrowd + Massachusetts, "AK" for AIcrowd + Kaggle
        # Data loaders
        batch_size=4,
        train_size=0.8,
        val_size=0.2,
        # Model
        model_name="ResNet50",
        model_pretrained=False,
        model_save_name="models/checkpoints/deeplabv3_resnet50_HUGE_retrain_2.pt",
        # Optimizer
        optim_name="adam",  # sgd
        optim_lr=3e-5,
        optim_momentum=0.9,  # TODO: Try with (optim_momentum != 0) and without (momentum = 0)?
        # Training
        n_steps=3000,
        eval_freq=100,
        # Wandb logging
        wandb_project="road-segmentation-clahe",
        wandb_run="deeplabv3-resnet50",
        entity="feeit",
    )
)

In [ ]:
# set seeds
set_seeds()

In [ ]:
# prepare train and validation loaders
train_loader, val_loader = prepare_data(args)

In [ ]:
# prepare model
model = prepare_model(args)
# define loss function
criterion = torch.nn.BCEWithLogitsLoss()
# prepare optimizer
optimizer = prepare_optimizer(model, args)

In [ ]:
MODEL = "models/checkpoints/deeplabv3_resnet50_HUGE.pt"
model.load_state_dict(torch.load(MODEL))

In [ ]:
# device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# train
trained_model = train(
    model, device, train_loader, val_loader, criterion, optimizer, args
)

Submission

In [ ]:
import torch
import torchvision.transforms.v2 as transforms
import torchvision.models.segmentation as models
import matplotlib.pyplot as plt
from datasets.TestDataset import TestDataset
from models.DeepLabV3 import ResNet50
from examples.mask_to_submission import *

In [ ]:
# select checkpoint
MODEL = "models/checkpoints/deeplabv3_resnet50_HUGE_retrain.pt"
# load checkpoint
try:
    checkpoint = torch.load(MODEL)
except:
    print("Loading checkpoint failed. Trying to load it with map_location.")
    checkpoint = torch.load(MODEL, map_location=torch.device("cpu"))
# create model
model = ResNet50()
# load model weights
model.load_state_dict(checkpoint)

In [ ]:
# path to the test folder
test_folder = "datasets/ethics/"

# set mean and std
mean = [0.3353, 0.3328, 0.2984]
std = [0.1967, 0.1896, 0.1897]
mean = [0.5616, 0.4848, 0.4514]
std = [0.1882, 0.1744, 0.1598]
# define transformations
transform = transforms.Compose(
    [
        transforms.Resize((608, 608)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)

# create test dataset
test_dataset = TestDataset(test_folder, transform=transform)

In [ ]:
from postprocessing import (
    apply_morphological_operations,
)

model.eval()
# set threshold
THR = 0.6
# create folder predictions if it doesn't exist
if not os.path.exists("predictions"):
    os.makedirs("predictions")
# save predictions
prediction_filenames = []
for i in range(len(test_dataset)):
    # get image
    image = test_dataset[i]
    # create prediction
    prediction = model(image.unsqueeze(0))
    # threshold prediction
    prediction = torch.sigmoid(prediction)
    # prediction = (prediction > THR).float()
    prediction = prediction.squeeze().detach().numpy()
    # apply morphological operations
    # prediction = apply_morphological_operations(prediction)
    # prediction = (prediction > THR).astype(int)
    # save prediction
    prediction_filename = "postprocess/prediction_" + str(i + 1) + ".png"
    prediction_filenames.append(prediction_filename)
    # upscale with bilinear interpolation to 608x608
    plt.imsave(prediction_filename, prediction.squeeze(), cmap="gray")

In [ ]:
# create submission
masks_to_submission("submission.csv", *prediction_filenames)